# Rogue Trader: path differences

## Intro

### Libraries

In [485]:
import pandas as pd
import numpy as np
import json
import os
import re
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
warnings.filterwarnings('ignore')
import openpyxl
import xlsxwriter

### Functions

In [486]:
def calculate_differences(df_old, df_new, item_name_col='Item_Name', column_group=None):
    """The function calculates differences between two dataframes.
    Parameters: 
    df_old - the previous table version
    df_new - the latest table version
    item_name_col - columns with item_names or ability_names or talent_names
    column_group - specify columns for the parameter "on" in merge function.
    
    """
    _diff = df_old.merge(df_new, indicator='Version',on = column_group, how = 'left',).loc[lambda x:x['Version']!='both']
    unique_names = _diff[item_name_col].unique().tolist() # list of unique names with differences
    index_values = _diff.index.values        
    try:
        _changes = (pd.concat([_diff.loc[index_values], df_new.loc[index_values]])
                            .reset_index()
                            .sort_values(by='index')                           
                        )
        # sort rows by name, previous version first
        _changes = _changes.rename(columns = {'index':'Item_Index'})
        _changes['Version'] = _changes['Version'].astype('str')
        _changes['Version'] = _changes['Version'].replace({'left_only':0,'nan':1})
        _changes = _changes.sort_values(by = [item_name_col, 'Item_Index','Version']).reset_index(drop = True)
        _changes['Version'] = _changes['Version'].replace({0:'Previous', 1:'Latest'})
    except KeyError as err:    # if keys are not found in the new table:  
                    print(f'Unexpected {err=}, {type(err)=}')
                    keys = re.findall('\d+',str(err))
                    keys = [int(i) for i in keys] # list of keys in key error message
                    index_values = [i for i in index_values if i not in keys]    # list of indexes not caused error                                    
                    # create   a common part of the table of differences
                    ## select rows of df_new with indexes in index_values and item_names in _diff['Item_Name].unique()
                    _subset = df_new[np.logical_and(df_new.index.isin(index_values), df_new[item_name_col].isin(unique_names))]
                    _changes = pd.concat([_diff.loc[index_values], _subset])                        
                    # find names of items caused error in the _diff table
                    names = _diff.loc[keys][item_name_col].unique().tolist()           
                    # select rows with those items from the df_new by name (instead of by indexes)
                    _temp = df_new[df_new[item_name_col].isin(names)]
                    _temp =pd.concat([_diff.loc[keys], _temp]) # create a problem part of the table of differences
                    #  create a one table
                    _changes = (pd.concat([_changes, _temp])
                            .reset_index()                            
                            )
                    # sort rows by name, previous version first
                    _changes = _changes.rename(columns = {'index':'Item_Index'})
                    _changes['Version'] = _changes['Version'].astype('str')
                    _changes['Version'] = _changes['Version'].replace({'left_only':0,'nan':1})
                    _changes = _changes.sort_values(by = [item_name_col, 'Version','Item_Index']).reset_index(drop = True)
                    _changes['Version'] = _changes['Version'].replace({0:'Previous', 1:'Latest'})                           
                    
    ## Do  some column arranging
    _changes = _changes.drop('Item_Index', axis = 1)
    if column_group is not None:
        column_group.insert(1,'Version')
        #column_group.insert(2,'Item_Index')
        #columns = _changes[column_group].columns.to_list()
        #columns = columns[1:2] +columns[0:1] + columns[-1:] +columns[2:-1]
        _changes = _changes[column_group]
        return _changes
    else:
        columns = _changes.columns.to_list()
        columns = columns[0:1] + columns[-1:] +  columns[1:-1]
        _changes = _changes[columns]
        return _changes

### Folders

In [487]:
#folders
folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables\Differences'


## Work part

### Weapons

In [488]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name','Item_Location', 'Family',	'Is_Heavy',	'Holding_Type',	'Origin',
	'Rarity',	'Damage_Type',	'Dodge_Penetration',
	'Rate_Of_Fire',	'Min_Base_Damage',	'Max_Base_Damage',	'Max_Ammo',	'Max_Distance',
    	'Range',	'Penetration',	'Recoil',	'Damage_Stat_Bonus_Factor',
        	'Ability_1_Name',	'Ability_1_Descr',	'Ability1_AP_Cost',	'Ability1_Type',
            	'Ability_2_Name',	'Ability_2_Descr',	'Ability2_AP_Cost',	'Ability2_Type',
                	'Ability_3_Name',	'Ability_3_Descr',	'Ability3_AP_Cost',	'Ability3_Type',
                    	'Ability_4_Name',	'Ability_4_Descr',	'Ability4_AP_Cost',	'Ability4_Type',
	'Ability_5_Name',	'Ability_5_Descr',	'Ability5_AP_Cost',	'Ability5_Type','Add_Descr']

In [489]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Weapons').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Weapons').fillna('')
changes_weapons = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_weapons.head())

Item_Name   Version  \
0   Ancient Plasma Rifle  Previous   
1   Ancient Plasma Rifle    Latest   
2  Anti-Vehicle Revolver  Previous   
3  Anti-Vehicle Revolver    Latest   
4      Assassin's Pistol  Previous   

                                                                                                              Item_Descr  \
0                                                                                 Deals an additional +(2 × INT) damage.   
1                                                                           Deals an additional +(2 × INT bonus) damage.   
2  On a successful hit, the shot explodes, dealing 20 impact damage to everyone in the area of effect around the target.   
3  On a successful hit, the shot explodes, dealing 20 impact damage to everyone in the area of effect around the target.   
4                                                                   This pistol deals double damage to adjacent enemies.   

          Vendor_Name Item_Location            Family  Is_Heavy Holding_Type  \
0  ExploratorChapter4                          Plasma     False    TwoHanded   
1  ExploratorChapter4                          Plasma     False    TwoHanded   
2      PirateChapter4                Solid projectile     False    OneHanded   
3      PirateChapter4                Solid projectile     False    OneHanded   
4                                    Solid projectile     False    OneHanded   

           Origin  Rarity Damage_Type  Dodge_Penetration  Rate_Of_Fire  \
0  RangedWeaponry  Unique      Energy                  0             0   
1  RangedWeaponry  Unique      Energy                  0             0   
2  RangedWeaponry  Unique      Impact                  0             3   
3  RangedWeaponry  Unique      Impact                  0             3   
4  RangedWeaponry  Unique    Piercing                  0             6   

   Min_Base_Damage  Max_Base_Damage  Max_Ammo  Max_Distance   Range  \
0               30               45        16            20  Ranged   
1               30               45        16            20  Ranged   
2               20               33         6            12  Ranged   
3               20               33         6            12  Ranged   
4               10               16        12            10  Ranged   

   Penetration  Recoil Damage_Stat_Bonus_Factor      Ability_1_Name  \
0           30      30                      One  Plasma Single Shot   
1           30      30                      One  Plasma Single Shot   
2           45      15                      One         Pistol Shot   
3           45      15                      One         Pistol Shot   
4           15      15                      One         Pistol Shot   

                            Ability_1_Descr Ability1_AP_Cost Ability1_Type  \
0  Single shot with equipped ranged weapon.              1.0    SingleShot   
1  Single shot with equipped ranged weapon.              1.0    SingleShot   
2                                                        1.0    SingleShot   
3                                                        1.0    SingleShot   
4                                                        1.0    SingleShot   

                    Ability_2_Name  \
0  Plasma Overcharge (Area Attack)   
1  Plasma Overcharge (Area Attack)   
2                     Pistol burst   
3                     Pistol Burst   
4                     Pistol burst   

                                                  Ability_2_Descr  \
0  Powerful area attack that overheats plasma weapon for 2 turns.   
1  Powerful area attack that overheats plasma weapon for 2 turns.   
2                                Burst fire with equipped pistol.   
3                                Burst fire with equipped pistol.   
4                                Burst fire with equipped pistol.   

  Ability2_AP_Cost Ability2_Type                    Ability_3_Name  \
0              2.0           AOE  Plasma Overcharge (Extra Damage)   
1       

#find differences sometines not working
diff_weapons = pd.concat([weapons_new,weapons_old]).drop_duplicates(keep = False)
display(diff_weapons.head())

### Armors

In [490]:
subset = ['Item_Name',	'Item_Descr', 'Vendor_Name', 'Item_Location',	'Origin',	'Rarity',
	'Category',	'DamageAbsorption',	'DamageDeflection',	'ProficiencyGroup']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Armors').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Armors').fillna('')
changes_armors = calculate_differences(df_old = df_old,
                                        df_new = df_new,
                                        column_group = subset)
display(changes_armors)

Unexpected err=KeyError('[103, 104] not in index'), type(err)=<class 'KeyError'>


Item_Name   Version  \
0           Adaptive Flak Vest  Previous   
1           Adaptive Flak Vest    Latest   
2                Loyalist Garb  Previous   
3                Loyalist Garb    Latest   
4                 Scout Armour  Previous   
5  Wraithbone Woven Battlesuit  Previous   
6  Wraithbone Woven Battlesuit    Latest   

                                                                                                                                                                                                                                                                                      Item_Descr  \
0     This flak vest grants the wearer +3 deflection and +15% armour at the beginning of every combat. During the first three rounds, the bonus deflection is reduced by −1 and armour by −5% at the end of each round.\nBase armour property: +10% armour if wearers Intellect is more than 55.   
1  This flak vest grants the wearer +3 deflection and +15% armour at the beginning of every combat. During the first three rounds, the bonus deflection is reduced by −1 and armour by −5% at the end of each round.\nBase armour property: +10% armour if wearers Intelligence is more than 55.   
2                                              If the wearer hasn't moved during their turn, their cover efficiency is increased by +20% for 1 round. If the wearer moves at least 3 cells per turn, they gain +20% dodge for 1 round.\nBase armour property: +20% dodge against ranged attacks.   
3                                             If the wearer has not moved during their turn, their cover efficiency is increased by +20% for 1 round. If the wearer moves at least 3 cells per turn, they gain +20% dodge for 1 round.\nBase armour property: +20% dodge against ranged attacks.   
4                                                                                                            The wearer's cover efficiency is increased by +15%. This bonus doubles if the wearer hasn't moved during their turn.\nBase armour property: +15% armour if wearer is in full cover.   
5                                                                 This armour restores (TGH bonus / 2) wounds each round when the wearer has any number of the following damage-over-time conditions: burning, toxin, bleeding.\nBase armour property: +10% Armour against non-Drukhari enemies.   
6                                                                 This armour restores (TGH bonus / 2) wounds each round when the wearer has any number of the following damage-over-time conditions: burning, toxin, bleeding.\nBase armour property: +10% armour against non-Drukhari enemies.   

       Vendor_Name                                Item_Location   Origin  \
0                                      Kiava Gamma Manufactorum  Armours   
1                                      Kiava Gamma Manufactorum  Armours   
2                   Planet: Dargonus, project: Schola Progenium  Armours   
3                   Planet: Dargonus, project: Schola Progenium  Armours   
4  DrusianChapter4                                               Armours   
5                                         Reaving Tempest Spire     Xeno   
6                                         Reaving Tempest Spire     Xeno   

   Rarity Category  DamageAbsorption  DamageDeflection ProficiencyGroup  
0  Unique   Medium                40                 0                   
1  Unique   Medium                40                 0                   
2  Unique    Light                50                 0                   
3  Unique    Light                50                 0                   
4  Unique   Medium                60                 0                   
5  Unique   Medium                50                 3                   
6  Unique   Medium                50                 3

### Shoulders

In [491]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Shoulders').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Shoulders').fillna('')
changes_shoulders = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_shoulders.head())

Item_Name   Version  \
0    Arch-Militant Hood  Previous   
1  Arch-Militant's Hood    Latest   
2       Cloak of Hatred  Previous   
3       Cloak of Hatred    Latest   
4    Holo-Field Emitter  Previous   

                                                                                                                                                                                                                                                                               Item_Descr  \
0                                                                                                                                        Each of the wearer's critical hits made with a ranged weapon increases their melee weapon critical damage by +STR bonus until the end of combat.   
1                                                                                                                                        Each of the wearer's critical hits made with a ranged weapon increases their melee weapon critical damage by +STR bonus until the end of combat.   
2                                                                                                                                                                           Each enemy killed by the wearer's area attacks increases damage of the wearer by +3% until the end of combat.   
3                                                                                                                                                                            Each enemy killed by the wearer's area attacks increases the wearer's damage by +3% until the end of combat.   
4  The wielder gains 3 charges of a holo-field at the start of combat. Whenever the wielder suffers damage, 1 charge of the holo-field is removed to prevent this damage.\nIf there are no charges left, damage isn't prevented and the wielder becomes staggered until the end of combat   

          Vendor_Name Item_Location         Origin  Rarity  
0                                          Textile  Common  
1                                          Textile  Common  
2     DrusianChapter2                        Chaos  Unique  
3     DrusianChapter2                        Chaos  Unique  
4  ExploratorChapter4                Miscellaneous  Common

### Feet

In [492]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Feet').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Feet').fillna('')
changes_feet = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_feet.head())

Item_Name   Version  \
0            Previous   
1              Latest   
2            Previous   
3              Latest   
4            Previous   

                                                                                                                                                                       Item_Descr  \
0   If the wearer didn't spend MP during their turn, they double their cover efficiency and gain immunity to the forced movement effects until the end of the wearer's next turn.   
1  If the wearer did not spend MP during their turn, they double their cover efficiency and gain immunity to the forced movement effects until the end of the wearer's next turn.   
2   If the wearer didn't spend MP during their turn, they double their cover efficiency and gain immunity to the forced movement effects until the end of the wearer's next turn.   
3  If the wearer did not spend MP during their turn, they double their cover efficiency and gain immunity to the forced movement effects until the end of the wearer's next turn.   
4   If the wearer didn't spend MP during their turn, they double their cover efficiency and gain immunity to the forced movement effects until the end of the wearer's next turn.   

       Vendor_Name Item_Location Origin  Rarity  
0  DrusianChapter2                 Tech  Unique  
1  DrusianChapter2                 Tech  Unique  
2  DrusianChapter4                 Tech  Unique  
3  DrusianChapter4                 Tech  Unique  
4  DrusianChapter4                 Tech  Unique

### Gloves

In [493]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Gloves').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Gloves').fillna('')
changes_gloves = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_gloves.head())

Item_Name   Version  \
0      Kasrkin Gauntlets  Previous   
1      Kasrkin Gauntlets    Latest   
2    Stabilising Bracers  Previous   
3    Stabilising Bracers    Latest   
4  Stabilising Gauntlets  Previous   

                                                                                                                                              Item_Descr  \
0                 These gauntlets grant the wearer a +10% bonus to critical hit chance. This bonus doubles if the wearer hasn't moved during their turn.   
1                These gauntlets grant the wearer a +10% bonus to critical hit chance. This bonus doubles if the wearer has not moved during their turn.   
2                                                   When the wearer uses Run and Gun, they gain a +(АGI bonus / 2) bonus damage on their ranged attacks.   
3                                                           When the wearer uses Run and Gun, they gain +(AGI bonus / 3) damage on their ranged attacks.   
4  When the wearer uses Run and Gun, they gain a +(АGI bonus / 2) bonus damage on their ranged attacks, and Run and Gun does not inflict winded on them.   

                 Vendor_Name Item_Location Origin  Rarity  
0  KasballicaMissionChapter2                 Tech  Common  
1  KasballicaMissionChapter2                 Tech  Common  
2         ExploratorChapter1                 Tech  Unique  
3         ExploratorChapter1                 Tech  Unique  
4                                    Chasm   Tech  Unique

### headgear

In [494]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Headgear').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Headgear').fillna('')
changes_headgear = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_headgear.head())

Item_Name   Version  \
0      Commander's Monocle  Previous   
1      Commander's Monocle    Latest   
2      Tiara Of Immaterium  Previous   
3  Tiara of the Immaterium    Latest   
4      Warp Seer's Goggles  Previous   

                                                                                                                                                                   Item_Descr  \
0  Whenever an ally under the effect of the wearer's Voice Of Command ability makes an attack during an extra turn, this attack deals an additional +(3 × FEL bonus)% damage.   
1  Whenever an ally under the effect of the wearer's Voice of Command ability makes an attack during an extra turn, this attack deals an additional +(3 × FEL bonus)% damage.   
2                                                                                                           The wearer gains +10 Willpower and suffers −50% less warp damage.   
3                                                                                                           The wearer gains +10 Willpower and suffers −50% less warp damage.   
4                                                                                                                                               Grants +Bonus WP Lore (Warp).   

  Vendor_Name                         Item_Location   Origin  Rarity  
0                                   Footfall Atrium     Tech  Unique  
1                                   Footfall Atrium     Tech  Unique  
2              Chapter 4, Quetza Temer, Laboratory.  Textile  Common  
3                                                    Textile  Common  
4                                                       Tech  Unique

### Necklaces

In [495]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Necklaces').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Necklaces').fillna('')
changes_neck = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_neck.head())

Item_Name   Version  \
0   Camaraderie Badge  Previous   
1   Camaraderie Badge    Latest   
2  Chartist's Pendant  Previous   
3  Chartist's Pendant    Latest   
4    Conmilitum Badge  Previous   

                                                                                                                                                                                                               Item_Descr  \
0                                                                                               When the wearer hits an enemy in melee range, the wearer's nearest ally gains (wearer's WP + FEL bonus) temporary wounds.   
1                                                                                         When the wearer hits an enemy in melee range, the wearer's nearest ally gains (wearer's WP bonus + FEL bonus) temporary wounds.   
2  The wearer gains +5 Fellowship and +10 Commerce.\nIf the wearer has Dogmatic — Adherent or above, their Lore (Imperium) will depend on Fellowship instead of Intelligence.\nThe wearer also gains +10 Lore (Imperium).   
3           The wearer gains +5 Fellowship and +10 Commerce.\nIf the wearer has Dogmatic — Adherent or above, their Lore (Imperium) will depend on Fellowship instead of Intelligence, and they gain +10 Lore (Imperium).   
4               When the wearer hits an enemy in melee range, all allies adjacent to the wearer gain (wearer's WP + FEL bonus) temporary wounds and perform an attack of opportunity against the enemy hit by the wearer.   

       Vendor_Name            Item_Location         Origin  Rarity  
0  DrusianChapter2                           Miscellaneous  Common  
1  DrusianChapter2                           Miscellaneous  Common  
2                   Governor Vyatt's Estate  Miscellaneous  Common  
3                   Governor Vyatt's Estate  Miscellaneous  Common  
4                                                     Tech  Unique

### Rings

In [496]:
subset = ['Item_Name', 'Item_Descr', 'Vendor_Name', 'Item_Location', 'Origin', 'Rarity']
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Rings').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Rings').fillna('')
changes_rings = calculate_differences(df_old = df_old, df_new = df_new, column_group=subset)
display(changes_rings.head())

Item_Name   Version  \
0          Accuser of Sin  Previous   
1          Accuser of Sin    Latest   
2   Bead Of Concentration  Previous   
3           Bead Of Might  Previous   
4  Burning Blood Talisman    Latest   

                                                                                                                                                                                                                                                                   Item_Descr  \
0                                                                                                                                                                           The Charge ability grants additional damage equal to wearer's WP bonus against daemons and xenos.   
1                                                                                                                                                                           The Charge ability grants additional damage equal to wearer's WP bonus against daemons and xenos.   
2                                                                                                                                                      The wearer gains an additional +1 AP for each consecutive turn that the wearer has not spent MP. Stacks up to 5 times.   
3                                                                                                                                                                If the wearer has 8 or more AP at the start of their turn, they gain +30% damage for the rest of the combat.   
4  This talisman grants the wearer +(wearer's WP bonus + 5)% dodge but reduces Fellowship by −30.\nIf the wearer has Heretical — Votary, they do not suffer the Fellowship penalty from the talisman and gains an additional +(wearer's WP bonus + 5)% damage against humans.   

          Vendor_Name                                      Item_Location  \
0                       Planet: Foulstone, project: In Drusus' Footsteps   
1                      Planet: Foulstone, project: In Drusus's Footsteps   
2  ExploratorChapter2                                                      
3                                                                  Chasm   
4                                                                          

    Origin  Rarity  
0     Tech  Unique  
1     Tech  Unique  
2  Jewelry  Unique  
3  Jewelry  Unique  
4    Chaos  Unique

### Class abilities

In [497]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Class abilities').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Class abilities').fillna('')
changes_class_abilities = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Ability_Name' )
display(changes_class_abilities)

Unexpected err=KeyError('[642, 644, 645, 646, 649, 651, 663, 664] not in index'), type(err)=<class 'KeyError'>


Class_Name   Version             Ability_Type         Ability_Name  \
0                 AP  Previous              AP Increase          AP Increase   
1      Arch-Militant  Previous                   Talent  Accustomed to Glory   
2      Arch-Militant    Latest                   Talent  Accustomed to Glory   
3            Officer  Previous            ActiveAbility     Air of Authority   
4            Officer    Latest            ActiveAbility     Air of Authority   
..               ...       ...                      ...                  ...   
179           Psyker  Previous                   Talent           Warp Speed   
180           Psyker    Latest                   Talent           Warp Speed   
181  Characteristics  Previous  Characteristic Increase      Weapon Skill +5   
182  Characteristics    Latest  Characteristic Increase      Weapon Skill +5   
183           Psyker  Previous                   Talent  Word of the Emperor   

                                                                                                                                                                                                                                         Ability_Descr  \
0                                                                                                                                                                                                            Grants +1 additional AP on the 20th rank.   
1                                                                                        Whenever the Arch-Militant or their ally uses a heroic act or a desperate measure, the Arch-Militant gains +2 damage for all attacks until the end of combat.   
2                                                                                Whenever the Arch-Militant or their ally uses a heroic act or a desperate measure, the Arch-Militant gains +1 and +10% damage on all attacks until the end of combat.   
3                            The Officer increases an ally's resolve by +(2 + Officer's resolve / 2) until the end of combat.\nIf the target is under the effect of Voice of Command, they will start their next turn with +1 additional action point.   
4                            The Officer increases an ally's resolve by +(2 + Officer's resolve / 3) until the end of combat.\nIf the target is under the effect of Voice of Command, they will start their next turn with +1 additional action point.   
..                                                                                                                                                                                                                                                 ...   
179                                                                                        Until the end of combat, the target ally increases their MP by +(3 + psyker's psy rating), their Agility by +(3 + psyker's psy rating), and their AP by +1.   
180                                                                                                    Until the end of combat, the target ally increases their MP by +(3 + psyker's psy rating),\ntheir Agility by +AgilityBonus, and their AP by +1.   
181  Weapon Skill (WS) reflects a character's competence in all forms of close combat. It increases their parry and reduces enemy parry and dodge chance against melee attacks. It also increases chances of scoring a critical hit in a melee combat.   
182              Weapon Skill (WS) reflects a character's competence in all forms of close combat. It increases their parry and reduces enemy parry chance against melee attacks. It also increases chances of scoring a critical hit in melee combat.   
183                                                                       All allies in a circle with a 5-cell radius gain +(1 + psy rating) resolve until the end of combat. Every additional stack of this effect increases the resolve bonus by +2.   

    Ability_Tag_0 Ability_Tag_1 Ability_Tag_2 Ab

### Homeworld talents

In [498]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='Homeworld talents').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='Homeworld talents').fillna('')
changes_hw_talents = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Talent' )
display(changes_hw_talents.head())

Talent   Version     World  \
0         Doing My Part  Previous  Imperial   
1         Doing My Part    Latest  Imperial   
2  Never Stop Believing  Previous  Fortress   
3  Never Stop Believing    Latest  Fortress   

                                                                                                                                                                                                                                    Description  
0                                                                             If no other character in the current party has the same archetype as this character, the first use of any ability from this archetype in combat costs −1 AP less.  
1                                                                      If no other character in the current party has the same archetype as this character, the first archetype ability used in each combat by this character costs −1 AP less.  
2  When all stacks of never stop shooting should be lost, they become (WP / 2) number of stacks instead.\nAdditionally, a fortress world character may upgrade their Willpower characteristic even if their current archetypes do not allow it.  
3                    When all stacks of never stop shooting should be lost, they become  stacks instead.\nAdditionally, a fortress world character may upgrade their Willpower characteristic even if their current archetypes do not allow it.

### Enemy ships

df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='EnemyShips').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='EnemyShips').fillna('')
changes_enemy_ships = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Ship_Name' )
display(changes_enemy_ships)

## Saving

save_folder = r'e:\Steam\RogueTrader\SaveFolder'
changes_weapons.to_excel(save_folder + r'\changes_weapons_test.xlsx', index= None)

In [499]:
# Save folder of ready tables
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
# excel writer
writer = pd.ExcelWriter(save_folder + r'\rogue_trader_patch_diff.xlsx')
# excel workbook
workbook = writer.book

Style format options:

In [500]:
#headers style
header_format = workbook.add_format({
        'bold':True,
        'text_wrap':True,
        'valign':'top',
        'align':'center',
        'fg_color':'#d5c4a5'
    })
# cells style
cell_names = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter',
                                       'bold':True})

cell_format = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter'})
cell_format2 = workbook.add_format({'align':'center',
                                       'valign':'vcenter'})
# Apply conditional formatting
cell_red = workbook.add_format({'bg_color':'#9C0006',
                                'align':'center',
                                 'valign':'vcenter' })
cell_light_red = workbook.add_format({'bg_color':'#e06666',
                                'align':'center',
                                 'valign':'vcenter' })
cell_yellow = workbook.add_format({'bg_color':'#eece6c',
                                'align':'center',
                                 'valign':'vcenter' })
cell_gray = workbook.add_format({'bg_color':'#999999',
                                'align':'center',
                                 'valign':'vcenter' })
#
cell_green = workbook.add_format({'bg_color':'#b6d7a8',
                                'align':'center',
                                 'valign':'vcenter' })
cell_blue = workbook.add_format({'bg_color':'#6fa8dc',
                                'align':'center',
                                 'valign':'vcenter' })

##### Weapons table

In [501]:
# WEAPONS
changes_weapons.to_excel(writer, sheet_name='Weapons_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Weapons_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_weapons.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:F',80/x, cell_format)
worksheet.set_column('G:G',100/x, cell_format2)
worksheet.set_column('H:H',130/x, cell_format2)
worksheet.set_column('I:I',130/x, cell_format2)
worksheet.set_column('J:U',100/x, cell_format2)
    ## ability 1 group
worksheet.set_column('V:V',85/x, cell_format)
worksheet.set_column('W:W',200/x, cell_format)
worksheet.set_column('X:X',80/x, cell_format2)
worksheet.set_column('Y:Y',92/x, cell_format2)
    ## ability 2 group
worksheet.set_column('Z:Z',85/x, cell_format)
worksheet.set_column('AA:AA',200/x, cell_format)
worksheet.set_column('AB:AB',80/x, cell_format2)
worksheet.set_column('AC:AC',92/x, cell_format2)
    ## ability 3 group
worksheet.set_column('AD:AD',85/x, cell_format)
worksheet.set_column('AE:AE',200/x, cell_format)
worksheet.set_column('AF:AF',80/x, cell_format2)
worksheet.set_column('AG:AG',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AH:AH',85/x, cell_format)
worksheet.set_column('AI:AI',200/x, cell_format)
worksheet.set_column('AJ:AJ',80/x, cell_format2)
worksheet.set_column('AK:AK',92/x, cell_format2)
    ## ability 5 group
worksheet.set_column('AL:AL',85/x, cell_format)
worksheet.set_column('AM:AM',200/x, cell_format)
worksheet.set_column('AN:AN',80/x, cell_format2)
worksheet.set_column('AO:AO',92/x, cell_format2)
# suppot columns
worksheet.set_column('AP:AP',200/x, cell_format)
    

0

### Armors

In [502]:
changes_armors.to_excel(writer, sheet_name='Armors_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Armors_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_armors.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)

## Armor stats columns
worksheet.set_column('F:K',100/x, cell_format2)

0

### Shoulders

In [503]:
changes_shoulders.to_excel(writer, sheet_name='Shoulders_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Shoulders_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_shoulders.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Feet

In [504]:
changes_feet.to_excel(writer, sheet_name='Feet_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Feet_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_feet.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Gloves

In [505]:
changes_gloves.to_excel(writer, sheet_name='Gloves_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Gloves_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_gloves.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Headgear

In [506]:
changes_headgear.to_excel(writer, sheet_name='Headgear_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Headgear_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_headgear.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Necklaces

In [507]:
changes_neck.to_excel(writer, sheet_name='Necklaces_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Necklaces_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_neck.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Rings

In [508]:
changes_rings.to_excel(writer, sheet_name='Rings_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Rings_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_rings.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

0

### Class abilities

In [509]:
changes_class_abilities.to_excel(writer, sheet_name='ClassAbilities_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['ClassAbilities_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_class_abilities.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',120/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',120/x, cell_format)
worksheet.set_column('D:D',120/x, cell_format)
worksheet.set_column('E:E',400/x, cell_format)
worksheet.set_column('F:I',120/x, cell_format)

0

### Homeworld abilities

In [510]:
changes_hw_talents.head(1)

Talent   Version     World  \
0  Doing My Part  Previous  Imperial   

                                                                                                                                                         Description  
0  If no other character in the current party has the same archetype as this character, the first use of any ability from this archetype in combat costs −1 AP less.

In [511]:
changes_hw_talents.to_excel(writer, sheet_name='HomeworldTalents_Patch_Changes', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['HomeworldTalents_Patch_Changes']

#Format header
for col_num, value in enumerate(changes_hw_talents.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',120/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',120/x, cell_format)
worksheet.set_column('D:D',400/x, cell_format)


0

In [512]:
writer.save()